In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.utils import resample
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
df = pd.read_csv('../Data/features-2017-04-25-v03.csv')

In [7]:
df.head() 

,is_duplicate,char_count1,char_count2,char_diff,word_count1,word_count2,word_diff,qword1,qword2,qword_diff,...,"pct_shared_(12.222, 24.444]","pct_shared_(24.444, 36.667]","pct_shared_(36.667, 48.889]","pct_shared_(48.889, 61.111]","pct_shared_(61.111, 73.333]","pct_shared_(73.333, 85.556]","pct_shared_(85.556, 97.778]","pct_shared_(97.778, 11]","pct_shared_(11, 122.222]",cosine
0,0,66,57,9,15,13,2,what,what,0,...,0,0,0,0,0,1,0,0,0,0.895532
1,0,51,88,37,11,16,5,what,what,0,...,0,0,1,0,0,0,0,0,0,0.527072
2,0,73,59,14,15,11,4,how,how,0,...,1,0,0,0,0,0,0,0,0,0.380873
3,0,50,65,15,13,19,6,why,when,1,...,0,0,0,0,0,0,0,0,0,0.000000
4,0,76,39,37,16,8,8,which,which,0,...,1,0,0,0,0,0,0,0,0,0.206084


In [9]:
# drop some features
df.drop(['char_count1', 'char_count2', 'char_diff', 'word_count1', 'word_count2', 'word_diff', 'qword1', 'qword2'], axis=1, inplace=True)

X = np.array(df.drop('is_duplicate', axis=1))
y = np.array(df['is_duplicate'].values.ravel())

In [10]:
def modelfit(model, features, label, scoring):
    ''' measure and report the model's performance
    
    parameters
    ----------
    model
        then model to evaluate
    features    array
        features
    label    array
        labels
    scoring    string
        the scoring method
    '''
    
    # fit the model
    model.fit(features, label)
    
    # make predictions
    y_pred = model.predict(features)
    y_proba = model.predict_proba(features)[:,1]
    
    # perform cross validation
    n_folds = 10
    seed = 7
    kfold = KFold(n_splits=n_folds, random_state=seed)
    cv_score = cross_val_score(model, features, label, cv=kfold, scoring=scoring)
    cv_accuracy = cross_val_score(model, features, label, cv=kfold, scoring='accuracy')
    
    # print the report
    print('\nModel Report')
    print('Model:\n', model)
    print('\nAccuracy: %.7g' % np.mean(cv_accuracy))
    print('\nCV score (%s): \nMean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g \n' % (scoring,np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    print('\nAUC score: %f' % metrics.roc_auc_score(y, y_proba ))
    fig = plt.figure()
    fig.suptitle('Cross Validation Score')
    ax = fig.add_subplot(111)
    plt.boxplot(cv_score)
    plt.show()

In [11]:
xgb0 = XGBClassifier()
modelfit(xgb0, X, y, 'neg_log_loss')

ValueError: could not convert string to float: '(61.111, 73.333]'

## 